Now after looking at the EDA and trying to understand the results, now we can move on to the machine learning model to get predictions for the future and see how good the information we obtained and cleaned is, for getting the number of available rooms in relation for each month of the year.

In [1]:
import pandas as pd
import pdb
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import datetime
from sklearn.linear_model import LinearRegression

This function splits our check-in date to different values so we get a column for each value

In [2]:
def split_checkin_to_date_columns(df): 
    day_date_array = [] 
    month_array = [] 
    year_array = [] 
    day_in_a_week_array = []
    # Monday is 0 in the .weekday() command, and Sunday is 6 there, so we convert it to the Israeli way of counting weekdays.
    # i.e - if we get 6 from the .weekday() command, it is Sunday so it should be 1 and not 6.
    week_days_js = {
            0 : 2,
            1 : 3,
            2 : 4,
            3 : 5,
            4 : 6, 
            5 : 7, 
            6 : 1 , 
        }
    
    # Create our lists of values so we can add them as columns to the dataframe afterwards.
    for part in df:
        date = datetime.datetime.strptime(part ,'%Y-%m-%d').date()
        day = date.day
        month = date.month
        year = date.year     
        day_in_week = week_days_js[datetime.date(day=day, month=month, year=year).weekday()]
        day_date_array.append(day)
        month_array.append(month)
        year_array.append(year)
        day_in_a_week_array.append(day_in_week)
          
    return day_in_a_week_array,day_date_array,month_array,year_array

This function adds the date's values we got from the split_checkin_to_date_columns function above, as columns into our dataframe.

In [3]:
def add_new_column_dates(df,day_in_a_week_array,day_date_array,month_array,year_array):
    df.insert(loc=1, column='day', value=day_in_a_week_array)
    df.insert(loc=2, column='date_day', value=day_date_array)
    df.insert(loc=3, column='month', value=month_array)
    df.insert(loc=4, column='year', value=year_array)
    
    return df2

This function gets the means for each column we have, and we create a new dataframe with these values.

In [44]:
def create_means_df(df2, month):
    
    ## https://stackoverflow.com/questions/65693824/pandas-get-column-average-for-rows-with-a-certain-value ## 
    hotel_price_mean = df2.groupby("hotel_name")["hotel_price"].mean()
    available_rooms_mean = df2.groupby("hotel_name")["available_rooms"].mean()
    hotel_rating_mean = df2.groupby("hotel_name")["hotel_rating"].mean()
    preferred_hotel_mean = df2.groupby("hotel_name")["preferred_hotel"].mean()
    nnumber_of_reviews_meas = df2.groupby("hotel_name")["number_of_reviews"].mean()
    distance_from_center_mean = df2.groupby("hotel_name")["distance_from_center"].mean()
    stars_count_mean = df2.groupby("hotel_name")["stars_count"].mean()
    new_hotel_mean = df2.groupby("hotel_name")["new_hotel"].mean()
    
    means_df = pd.DataFrame({
                "month": i,
                "hotel_name" : hotel_price_mean.keys(), 
                "available_rooms" : available_rooms_mean.values,
                "hotel_price" : hotel_price_mean.values,
                "hotel_rating" : hotel_rating_mean.values,
                "preferred_hotel" : preferred_hotel_mean.values, 
                "number_of_reviews" : nnumber_of_reviews_meas.values,
                "distance_from_center" : stars_count_mean.values, 
                "stars_count" : stars_count_mean.values,
                "new_hotel" : new_hotel_mean.values,
                ### Add 'day' as well !! And add to the Machine Learning too. ###
                })  
    return means_df

This function uses a Linear Regression model on the given month and its means values of our hotels' information in a relation to the available rooms in the said month. We train the model with 80% of the values, and test it with the other 20% left as it is usually done. We then fit the model and try to predict the values, then we print the information we got to check how good our model is.

In [45]:
def return_r_squared(df, month):
    print("Number of Rows: " + str(len(df)))
    print("Free Available Hotels: " + str(len(df[df['available_rooms'] == 10])))
    df.drop(df[df['available_rooms'] == 10].index, inplace = True)
    df = create_means_df(df, month)
    x = df[[
    'month',    
    'stars_count',
    'hotel_price',
    'number_of_reviews',
    'distance_from_center',
    'hotel_rating',
    'new_hotel',
    'preferred_hotel',    
     ]]

    # https://labs.vocareum.com/main/main.php?m=editor&asnid=537364&stepid=537365&hideNavBar=1 + https://towardsdatascience.com/machine-learning-simple-linear-regression-with-python-f04ecfdadc13 
    y = df['available_rooms']

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 10)

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    prediction = lr.predict(X_test)

    pred = pd.DataFrame({'Actual': y_test.tolist(), 'Predicted': prediction}).head(25)

    print("Slope:",lr.coef_)
    print("Intercept:",lr.intercept_)

    print("R2:",lr.score(x,y))

    return

Below is our main code, we will use all of our functions above to eventually print out our machine learning model's about the number of available rooms per each month of the year that we got according to our dataframe.

-> We need R2 results closer 1 or -1 rather than 0, as 1 means the most perfect linear relation, while -1 means the most perfect negative linear relation.

In [46]:
if __name__ == "__main__":
    df = pd.read_csv('./CsvFolder/Hotels365Day.csv')
    # We sort the values by check-in dates, then we reset the indexes and create a dates column to split it 
    # into 4 columns to add to our dataframe for better machine learning prediction.
    df.sort_values('check_in', inplace=True)
    df2 = df.iloc[:, 1:].reset_index(drop=True)
    dates_columns = df2['check_in']

    day_in_a_week_array,day_date_array,month_array,year_array = split_checkin_to_date_columns(dates_columns)
    
    ## This is the new data frame after splitting the date to 4 columns 
    df2 = add_new_column_dates(df2,day_in_a_week_array,day_date_array,month_array,year_array)
    
    # Remove hotels with values that are on edge (Usually new hotels which may ruin our prediction)
    df2.drop(df2[df2['hotel_rating'] < 3].index, inplace = True)
    df2.drop(df2[df2['number_of_reviews'] < 10].index, inplace = True)
    
    # Get the linear regression model information for each month in the next year.
    for i in range(1,13):
        print("Month Number "+ str(i))    
        return_r_squared(df2[df2['month'] == i], i)
        print('-----------------\n\n')

Month Number 1
Number of Rows: 432
Free Available Hotels: 81
Slope: [ 0.00000000e+00  1.20252340e-01 -4.26603271e-03 -3.31061271e-04
  1.20252340e-01  1.35024825e+00 -4.57921850e-01 -5.79474631e-01]
Intercept: -3.0720940275415662
R2: 0.8187466398111195
-----------------


Month Number 2
Number of Rows: 721
Free Available Hotels: 246
Slope: [ 0.00000000e+00  9.73628693e-04 -9.24085555e-04  3.75863364e-06
  9.73628693e-04 -3.21095187e-02 -1.06209160e+00 -3.34231936e-01]
Intercept: 5.0472335089384615
R2: 0.20356916945531478
-----------------


Month Number 3
Number of Rows: 1247
Free Available Hotels: 573
Slope: [ 0.00000000e+00  9.04517972e-02 -1.05154252e-03 -1.17573107e-04
  9.04517972e-02  4.33011813e-01 -3.10893305e-01 -8.98687548e-01]
Intercept: 1.9135847169014442
R2: 0.23820618131796745
-----------------


Month Number 4
Number of Rows: 886
Free Available Hotels: 370
Slope: [ 0.00000000e+00  3.63382221e-02 -2.59124606e-03 -3.66823308e-04
  3.63382221e-02  9.87163276e-01  3.28937313

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12700\2945371087.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['available_rooms'] == 10].index, inplace = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12700\2945371087.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['available_rooms'] == 10].index, inplace = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12700\2945371087.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

From the results, we can understand that our model is not very strong for the available rooms number per month of the year relation in most months, but for the months closest and farthest from the starting month of the information, it is much better than the months in the middle of it.